In [3]:
import pandas as pd
import os

def analyze_tickers(interval='1d'):
    # Directorio donde están los resultados de los análisis de cada ticker
    input_dir = 'nasdaq_analysis_results'
    # Directorio donde se guardará el resumen del análisis
    output_dir = 'analysis_results'

    # Crear el directorio de salida si no existe
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Lista para almacenar los tickers recomendados
    recommended_tickers = []
    print(f"Obtenemos lista de archivos de análisis disponibles en {input_dir}...")
    # Obtener la lista de archivos de análisis disponibles
    files = [f for f in os.listdir(input_dir) if f.endswith(f'{interval}_analysis.csv')]
    print('Ya se obtuvo la lista de archivos de análisis disponibles.')
    for file in files:
        ticker = file.split('_')[0]
        print(f"Procesando ticker: {ticker}")  # Imprime el ticker que se está procesando
        filepath = os.path.join(input_dir, file)

        # Leer los datos
        df = pd.read_csv(filepath, index_col=0, parse_dates=True)

        # Asegurarse de que los indicadores necesarios están presentes
        required_columns = ['momentum_rsi', 'trend_macd', 'trend_macd_signal', 'close']
        if not all(column in df.columns for column in required_columns):
            print(f"Indicadores necesarios no encontrados para {ticker}.")
            continue

        # Obtener los últimos valores de los indicadores
        latest_data = df.iloc[-1]

        # Criterios de compra basados en indicadores técnicos
        # Ejemplo:
        # - RSI por debajo de 30 (sobreventa)
        # - MACD cruza por encima de la señal (indicador alcista)
        rsi = latest_data['momentum_rsi']
        macd = latest_data['trend_macd']
        macd_signal = latest_data['trend_macd_signal']
        close_price = latest_data['close']

        # Señal de RSI
        rsi_signal = rsi < 30

        # Señal de MACD (cruce alcista)
        macd_signal_cross = False
        if len(df) >= 2:
            prev_macd = df['trend_macd'].iloc[-2]
            prev_macd_signal = df['trend_macd_signal'].iloc[-2]
            macd_signal_cross = (prev_macd < prev_macd_signal) and (macd > macd_signal)

        # Señal de cruce de medias móviles (ejemplo)
        sma_fast_col = 'trend_sma_fast'
        sma_slow_col = 'trend_sma_slow'
        sma_crossover = False
        if sma_fast_col in df.columns and sma_slow_col in df.columns:
            sma_fast = df[sma_fast_col]
            sma_slow = df[sma_slow_col]
            if len(df) >= 2:
                prev_sma_fast = sma_fast.iloc[-2]
                prev_sma_slow = sma_slow.iloc[-2]
                curr_sma_fast = sma_fast.iloc[-1]
                curr_sma_slow = sma_slow.iloc[-1]
                sma_crossover = (prev_sma_fast < prev_sma_slow) and (curr_sma_fast > curr_sma_slow)

        # Si se cumplen las señales, añadir a la lista de recomendados
        if rsi_signal or macd_signal_cross or sma_crossover:
            recommended_tickers.append({
                'Ticker': ticker,
                'Precio Actual': close_price,
                'RSI': rsi,
                'MACD': macd,
                'MACD_Signal': macd_signal,
                'RSI_Sobreventa': rsi_signal,
                'MACD_Cruce_Alcista': macd_signal_cross,
                'SMA_Cruce_Alcista': sma_crossover
            })

    # Convertir la lista de recomendados en DataFrame
    recommendations_df = pd.DataFrame(recommended_tickers)
    if not recommendations_df.empty:
        recommendations_df = recommendations_df.set_index('Ticker')

        # Guardar el DataFrame en un archivo CSV dentro del directorio output_dir
        output_file = os.path.join(output_dir, f'recomendaciones_{interval}.csv')
        recommendations_df.to_csv(output_file)
        print(f"\nAnálisis guardado en: {output_file}")

        # Mostrar los tickers recomendados
        print("\nTickers recomendados para compra basados en el análisis técnico:")
        print(recommendations_df)
    else:
        print("\nNo se encontraron tickers que cumplan los criterios de compra.")

In [4]:
analyze_tickers(interval='1d')

Obtenemos lista de archivos de análisis disponibles en nasdaq_analysis_results...
Ya se obtuvo la lista de archivos de análisis disponibles.
Procesando ticker: ADBE
Procesando ticker: AMD
Procesando ticker: ABNB
Procesando ticker: GOOGL
Procesando ticker: GOOG
Procesando ticker: AMZN
Procesando ticker: AEP
Procesando ticker: AMGN
Procesando ticker: ADI
Procesando ticker: ANSS
Procesando ticker: AAPL
Procesando ticker: AMAT
Procesando ticker: ASML
Procesando ticker: AZN
Procesando ticker: TEAM
Procesando ticker: ADSK
Procesando ticker: ADP
Procesando ticker: BKR
Procesando ticker: BIIB
Procesando ticker: BKNG
Procesando ticker: AVGO
Procesando ticker: CDNS
Procesando ticker: CDW
Procesando ticker: CHTR
Procesando ticker: CTAS
Procesando ticker: CSCO
Procesando ticker: CCEP
Procesando ticker: CTSH
Procesando ticker: CMCSA
Procesando ticker: CEG
Procesando ticker: CPRT
Procesando ticker: CSGP
Procesando ticker: COST
Procesando ticker: CRWD
Procesando ticker: CSX
Procesando ticker: DDOG
Pr